In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'
                       .format(postgres_user, postgres_pw, postgres_host,
                              postgres_port, postgres_db))

df = pd.read_sql_query('SELECT * FROM useducation', con=engine)

engine.dispose()

In [3]:
df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


In [12]:
cols = [x for x in df.columns if x not in ('PRIMARY_KEY', 'STATE', 'YEAR')]

In [6]:
states = df['STATE'].unique()

#interpolating for the columns that we can
for state in states:
    df.loc[df['STATE'] == state, cols] = df.loc[df['STATE'] == state, cols].interpolate()
    
#dropping columns with missing data that were not fixed via interpolation 
df.dropna(inplace=True)
print(df.isnull().sum())

PRIMARY_KEY                     0
STATE                           0
YEAR                            0
ENROLL                          0
TOTAL_REVENUE                   0
FEDERAL_REVENUE                 0
STATE_REVENUE                   0
LOCAL_REVENUE                   0
TOTAL_EXPENDITURE               0
INSTRUCTION_EXPENDITURE         0
SUPPORT_SERVICES_EXPENDITURE    0
OTHER_EXPENDITURE               0
CAPITAL_OUTLAY_EXPENDITURE      0
GRADES_PK_G                     0
GRADES_KG_G                     0
GRADES_4_G                      0
GRADES_8_G                      0
GRADES_12_G                     0
GRADES_1_8_G                    0
GRADES_9_12_G                   0
GRADES_ALL_G                    0
AVG_MATH_4_SCORE                0
AVG_MATH_8_SCORE                0
AVG_READING_4_SCORE             0
AVG_READING_8_SCORE             0
dtype: int64


1. Create a new score variable from the weighted averages of all score variables in the datasets. **Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!**.


In [7]:
df['w_score'] = (df['GRADES_4_G']*.5*(df['AVG_MATH_4_SCORE']+df['AVG_READING_4_SCORE']))+(df['GRADES_8_G']*.5*(df['AVG_MATH_8_SCORE']+df['AVG_READING_8_SCORE']))/(df['GRADES_4_G']+df['GRADES_8_G'])


2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?


In [11]:
df[['w_score','TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE']].corr()

,w_score,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
w_score,1.000000,0.919653,0.879631,0.922219,0.959375,0.932440
TOTAL_EXPENDITURE,0.919653,1.000000,0.990829,0.993175,0.946483,0.928576
INSTRUCTION_EXPENDITURE,0.879631,0.990829,1.000000,0.975867,0.907235,0.889763
SUPPORT_SERVICES_EXPENDITURE,0.922219,0.993175,0.975867,1.000000,0.953789,0.913680
OTHER_EXPENDITURE,0.959375,0.946483,0.907235,0.953789,1.000000,0.924197
CAPITAL_OUTLAY_EXPENDITURE,0.932440,0.928576,0.889763,0.913680,0.924197,1.000000


Capital outlay expenditure is the most correlated to our newly created weighted score variable. 

3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?


In [20]:
#long way
X = df[['TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE']]

#standardizing our data so that variables have mean oif 0 and std of 1
X = StandardScaler().fit_transform(X)

#transposing to be compatable with numpy
Xt = X.T

#using numpy to estimate covariance matrix
#Note this is now a square array, which is a requirement for the next step
Cx = np.cov(Xt)

#calculating eigenvalues and eigenvectors
eig_val_cov, eig_vec_cov = np.linalg.eig(Cx)


eig_val_cov[0]/eig_val_cov.sum()

0.9541168023970665

In [21]:
#short way

#Alternatively using PCA from SKlearn
sklearn_pca = PCA(n_components=1)
df['pca_1'] = sklearn_pca.fit_transform(X)

#calling the variance ratio to see 
sklearn_pca.explained_variance_ratio_

array([0.9541168])

In [19]:
#Examining shape of data as we transform 

print(df.shape)
print(X.shape)
print(Cx.shape)
print(eig_val_cov.shape)
print(eig_vec_cov.shape)

(1046, 27)
(1046, 6)
(6, 6)
(6,)
(6, 6)


We can see that 94.7% of the variance is explained by the first component 

4. What is the correlation between the overall score variable and the 1st principal component? 


In [22]:
df[['w_score', 'pca_1', 'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE']].corr()

,w_score,pca_1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
w_score,1.000000,0.944357,0.919653,0.879631,0.922219,0.959375,0.932440
pca_1,0.944357,1.000000,0.995210,0.975856,0.990651,0.968760,0.952869
TOTAL_EXPENDITURE,0.919653,0.995210,1.000000,0.990829,0.993175,0.946483,0.928576
INSTRUCTION_EXPENDITURE,0.879631,0.975856,0.990829,1.000000,0.975867,0.907235,0.889763
SUPPORT_SERVICES_EXPENDITURE,0.922219,0.990651,0.993175,0.975867,1.000000,0.953789,0.913680
OTHER_EXPENDITURE,0.959375,0.968760,0.946483,0.907235,0.953789,1.000000,0.924197
CAPITAL_OUTLAY_EXPENDITURE,0.932440,0.952869,0.928576,0.889763,0.913680,0.924197,1.000000


5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?


We saw that the first principal component captured 94% of the variance. It would be better to use Other expenditure instead because it is more correlated with overall score. 
